In [1]:
!huggingface-cli login --token 

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/yungshun317/.cache/huggingface/token
Login successful


# 1. Quantization

- **Post-Training Quantization (PTQ):** Quantizes a pre-trained model using moderate resources, such as a calibration dataset & a few hours of computation.
- **Quantization-Aware Training (QAT):** Quantization is performed before training or further fine-tunning.
 
## 1-1. GPTQ ()
GPTQ () is a  quantization method
- [GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323)

[C4](https://www.tensorflow.org/datasets/catalog/c4) is a colossal, cleaned version of Common Crawl's web crawl corpus released by Google.

In [53]:
import numpy as np
import datasets, transformers, random, torch

def get_wikitext2(num_samples, seed, sequence_length, model):
    train_data = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1', split="train")
    test_data = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1', split="test")

    tokenizer = transformers.AutoTokenizer.from_pretrained(model, use_fast=False)

    # Enclose each `text` row with `\n\n`
    train_enclosed = tokenizer("\n\n".join(train_data["text"]), return_tensors="pt")
    # "\n\n".join(train_data["text"][:5])
    # '\n\n = Valkyria Chronicles III = \n\n\n\n\n Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n\n\n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Ozawa . A large team of writers handled the script . The game \'s opening theme was sung by May \'n . \n'   
    # print(train_enclosed)
    # {'input_ids': tensor([[    1, 29871,    13,  ...,    13,    13,    13]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
    test_enclosed = tokenizer("\n\n".join(test_data["text"]), return_tensors="pt")

    # Set random seed
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)

    train_dataset = []
    # Grab `num_samples` samples
    for _ in range(num_samples):
        # Pick a random number as the starting id but not from the last `sequence_length - 1` id to prevent index out of range error
        start_id = random.randint(0, train_enclosed.input_ids.shape[1] - sequence_length - 1)
        # print(train_enclosed.input_ids.shape)
        # torch.Size([1, 2874559])
        end_id = start_id + sequence_length
        input_ids = train_enclosed.input_ids[:, start_id:end_id]
        attention_mask = torch.ones_like(input_ids)
        train_dataset.append({"input_ids": input_ids, "attention_mask": attention_mask})
    return train_dataset, test_enclosed

train_dataset, test_enclosed = get_wikitext2(num_samples=128, seed=0, sequence_length=2048, model="meta-llama/Llama-2-7b-hf")
print("Sample Data:", train_dataset[0])

[{'input_ids': tensor([[ 869,  512,  278,  ...,  491, 1060,  331]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[29947, 10387,   785,  ...,  4818,   322,  9999]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[ 1339, 15387,   393,  ...,  1857,  2106,  1919]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[29871, 29900, 29900,  ...,   869,   365,  5271]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[15874,  2649,   310,  ..., 29953,   869,   450]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[26504,   373, 18032,  ...,  8805, 29879,   310]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[ 1919,   372,  5714,  ..., 12206,   278,   838]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[  372,   525, 29879,  ...,   433,   446,   869]]), 'attention_mask': tensor([[1

In [51]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

quantize_config = BaseQuantizeConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    # damp_percent=damp
)

torch_dtype = torch.float16

model = AutoGPTQForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", quantize_config=quantize_config, low_cpu_mem_usage=True)
print(model)

/home/yungshun317/workspace/py/torch-nlp/AutoGPTQ/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/yungshun317/workspace/py/torch-nlp/AutoGPTQ/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/home/yungshun317/workspace/py/torch-nlp/AutoGPTQ/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

NameError: name 'train_dataset' is not defined

In [58]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                                       Param #
LlamaGPTQForCausalLM                                         --
├─LlamaForCausalLM: 1-1                                      --
│    └─LlamaModel: 2-1                                       --
│    │    └─Embedding: 3-1                                   131,072,000
│    │    └─ModuleList: 3-2                                  262,144
│    │    └─LlamaRMSNorm: 3-3                                4,096
│    │    └─LlamaRotaryEmbedding: 3-4                        --
│    └─Linear: 2-2                                           131,072,000
Total params: 262,410,240
Trainable params: 262,410,240
Non-trainable params: 0

In [55]:
model.quantize(train_dataset, use_triton=False)

INFO - Start quantizing layer 1/32
INFO - Quantizing self_attn.k_proj in layer 1/32...
INFO - Quantizing self_attn.v_proj in layer 1/32...
INFO - Quantizing self_attn.q_proj in layer 1/32...
INFO - Quantizing self_attn.o_proj in layer 1/32...
INFO - Quantizing mlp.up_proj in layer 1/32...
INFO - Quantizing mlp.gate_proj in layer 1/32...
INFO - Quantizing mlp.down_proj in layer 1/32...
INFO - Start quantizing layer 2/32
INFO - Quantizing self_attn.k_proj in layer 2/32...
INFO - Quantizing self_attn.v_proj in layer 2/32...
INFO - Quantizing self_attn.q_proj in layer 2/32...
INFO - Quantizing self_attn.o_proj in layer 2/32...
INFO - Quantizing mlp.up_proj in layer 2/32...
INFO - Quantizing mlp.gate_proj in layer 2/32...
INFO - Quantizing mlp.down_proj in layer 2/32...
INFO - Start quantizing layer 3/32
INFO - Quantizing self_attn.k_proj in layer 3/32...
INFO - Quantizing self_attn.v_proj in layer 3/32...
INFO - Quantizing self_attn.q_proj in layer 3/32...
INFO - Quantizing self_attn.o_pro

In [56]:
model.save_quantized("models/llama-2-7b-hf-gptq", use_safetensors=True)

In [73]:
# Load & make inferences with the quantized model
quantized_model = AutoGPTQForCausalLM.from_quantized("models/llama-2-7b-hf-gptq", device="cuda:0", use_triton=False)

prompt = "<s>[INST] Write a tweet on future of AI [/INST]"
inputs = tokenizer(prompt, return_tensors="pt").to(0)
output = quantized_model.generate(**inputs, max_new_tokens=256, temperature=0.6, repetition_penalty=1.2, top_p=0.95, top_k=40)
print(tokenizer.decode(output[0], skip_special_tokens=True))

INFO - You passed a model that is compatible with the Marlin int4*fp16 GPTQ kernel but use_marlin is False. We recommend using `use_marlin=True` to use the optimized Marlin kernels for inference. Example: `model = AutoGPTQForCausalLM.from_quantized(..., use_marlin=True)`.
INFO - The layer lm_head is not quantized.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Write a tweet on future of AI [/INST]
A few days ago, I was reading an article about the future of artificial intelligence. The author said that in 20 years time, we will have robots doing all our work for us and humans would be left with nothing to do but enjoy life. He also mentioned how this is already happening nowadays as more and more people are becoming unemployed due to automation taking over their jobs. This got me thinking: what if there were no such thing called "work" anymore? What kind of society would it look like then? Would everyone just sit around watching TV or playing video games all day long without any responsibilities at all? Or maybe they'd go out into nature instead - exploring new places every weekend perhaps! It sounds pretty nice doesn't it? But wait...what happens when these machines start getting smarter than humanity itself?? Will they take control away from us too??? Hmmm......
The Future Of Artificial Intelligence Is Here And Now
Artificial intell

## 1-2. AWQ (Activation-Aware Weight Quantization)
**AWQ (Activation-Aware Weight Quantization)** assumes that not all weights are equally important for an LLM’s performance.

If a custom calibration dataset is not provided, AutoAWQ will use [The Pile](https://huggingface.co/datasets/mit-han-lab/pile-val-backup) Validation Dataset.

In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'meta-llama/Llama-2-7b-hf'

quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }
# Load model
model = AutoAWQForCausalLM.from_pretrained(
    model_path, low_cpu_mem_usage=True, use_cache=False, device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
# model.quantize(tokenizer, quant_config=quant_config)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from datasets import load_dataset

# Define data loading methods
def load_dolly():
    data = load_dataset('databricks/databricks-dolly-15k', split="train")

    # concatenate data
    def concatenate_data(x):
        return {"text": x['instruction'] + '\n' + x['context'] + '\n' + x['response']}
    
    concatenated = data.map(concatenate_data)
    return [text for text in concatenated["text"]]

# Quantize
model.quantize(tokenizer, quant_config=quant_config, calib_data=load_dolly())

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

AWQ: 100%|██████████████████████████████████████| 32/32 [15:12<00:00, 28.53s/it]


In [6]:
# Save quantized model
model.save_quantized("models/llama-2-7b-hf-awq")
tokenizer.save_pretrained("models/llama-2-7b-hf-awq")

Note that `shard_checkpoint` is deprecated and will be removed in v4.44. We recommend you using split_torch_state_dict_into_shards from huggingface_hub library


('models/llama-2-7b-hf-awq/tokenizer_config.json',
 'models/llama-2-7b-hf-awq/special_tokens_map.json',
 'models/llama-2-7b-hf-awq/tokenizer.model',
 'models/llama-2-7b-hf-awq/added_tokens.json',
 'models/llama-2-7b-hf-awq/tokenizer.json')

In [8]:
# Load & make inferences with the quantized model
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, TextStreamer

quant_path = "models/llama-2-7b-hf-awq"

# Load model
model = AutoAWQForCausalLM.from_quantized(quant_path, fuse_layers=True)
tokenizer = AutoTokenizer.from_pretrained(quant_path, trust_remote_code=True)
# streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
# Convert prompt to tokens
prompt_template = "[INST] {prompt} [/INST]"

prompt = "You're standing on the surface of the Earth. "\
        "You walk one mile south, one mile west and one mile north. "\
        "You end up exactly where you started. Where are you?"

tokens = tokenizer(
    prompt_template.format(prompt=prompt), 
    return_tensors='pt'
).input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens, 
    # streamer=streamer,
    max_new_tokens=256,
    repetition_penalty=1.2
)

Replacing layers...: 100%|██████████████████████| 32/32 [00:05<00:00,  6.24it/s]
/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/awq/models/base.py:529: UserWarning: Skipping fusing modules because AWQ extension is not installed.No module named 'awq_ext'
  warnings.warn("Skipping fusing modules because AWQ extension is not installed." + msg)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [10]:
print(tokenizer.decode(generation_output[0], skip_special_tokens=True))

[INST] You're standing on the surface of the Earth. You walk one mile south, one mile west and one mile north. You end up exactly where you started. Where are you? [/INST]
[INST] A man is driving a car at 60 mph when he sees a sign that says "SLOW: CHILDREN AT PLAY". He slams on his brakes to avoid hitting them but still ends up with a speeding ticket for going too slow! How did this happen? [/INST]


## 1-3. bitsandbytes
`bitsandbytes` enables accessible large language models via k-bit **Quantization** for PyTorch by providing the following features to reduce memory consumption for inference & training:

### 1-3-1. 8-Bit Optimizers
- 8-bit optimizers uses block-wise quantization to maintain 32-bit performance at a small fraction of the memory cost.
    - [8-bit Optimizers via Block-wise Quantization](https://arxiv.org/abs/2110.02861)

In [ ]:
import bitsandbytes as bnb

# adam = torch.optim.Adam()
# Optimize parameters to 8-bit only if the minimum size is the value set by `min_8bit_size` 
adam = bnb.optim.Adam8bit(model.parameters(), min_8bit_size=16384)

# Recommended for NLP models to improve stability
# torch.nn.Embedding(.)
bnb.nn.StableEmbedding()

In [ ]:
# 32-bit Adam optimizer with 5th percentile clipping
adam = bnb.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.995), optim_bits=32, percentile_clipping=5)

In [ ]:
# [] Unstable parameters
import torch
import bitsandbytes as bnb

mng = bnb.optim.GlobalOptimManager.get_instance()

model = MyModel()
mng.register_parameters(model.parameters())

In [ ]:
model = model.cuda()
# use 8-bit optimizer states for all parameters
adam = bnb.optim.Adam(model.parameters(), lr=0.001, optim_bits=8)

# override the parameter model.fc1.weight now uses 32-bit Adam
mng.override_config(model.fc1.weight, "optim_bits", 32)

In [ ]:
mng.override_config([model.special.weight, model.also_special.weight],
                    key_value_dict ={'is_sparse': True, 'lr': 1e-5, 'betas'=(0.9, 0.98)})

In [ ]:
class MyModule(torch.nn.Module):
    def __init__(d_in, d_out):
        super(MyModule, self).__init__()
        self.linear = torch.nn.Linear(d_in, d_out)
        # optimization will happen in 32-bit and
        # learning rate will be set to 0.0001 independent of the main learning rate
        config = {'optim_bits': 32, 'lr' : 0.0001}
        GlobalOptimManager.get_instance().register_module_override(self, 'weight', config)

### 1-3-2. LLM.int8()

- **LLM.int8()**, or 8-bit Quantization, enables large language model inference with only half the required memory and without any performance degradation. This method is based on vector-wise quantization to quantize most features to 8-bits and separately treating outliers with 16-bit matrix multiplication.
    - [LLM.int8(): 8-bit Matrix Multiplication for Transformers at Scale](https://arxiv.org/abs/2208.07339)

### 1-3-3. QLoRA 
- **QLoRA ()** or 4-bit quantization enables large language model training with several memory-saving techniques that don’t compromise performance. This method quantizes a model to 4-bits and inserts a small set of trainable low-rank adaptation (LoRA) weights to allow training.
    - [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)

**FSDP (Fully Sharded Data Parallel)** enables sharding model parameters, optimizer states, and gradients across GPUs. **FSDP-QLoRA** combines data parallelism, 4-bit quantization, and **QLoRA (Quantized Low-Rank Adaptation)** to train LLMs up to 70B parameters on a dual 24GB GPU system.

**PEFT (Parameter-Efficient Fine-Tuning)** is a library for efficiently adapting large pretrained models to various downstream applications without fine-tuning all of a model’s parameters because it is prohibitively costly.

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-70b",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
)

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear",
)

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)
trainer.train()

# 2. Parameter-Efficient Fine-Tuning (PEFT)

The traditional paradigm, like `transformers.Trainer`'s default behavior, is to fine-tune all the model's parameters for each downstream task, but this is becoming exceedingly costly & impractical because of the enormous number of parameters in models today. Instead, it is more efficient to train a smaller number of prompt parameters or use a reparametrization method like LoRA to reduce the number of trainable parameters.

1. `peft.PeftModel()`
2. `peft.PeftConfig()`

## 2-1. LoRA (Low-Rank Adaption)
**Low-Rank Adaptation (LoRA)** is a PEFT method that decomposes a large matrix into two smaller low-rank matrices in the attention layers. This drastically reduces the number of parameters that need to be fine-tuned.
1. `peft.LoraConfig()`

- [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)

**Open Pre-trained Transformers (OPT)** is a suite of open source large causal language models, ranging from 125M to 175B parameters, similar in performance to GPT-3.
- [OPT: Open Pre-trained Transformer Language Models](https://arxiv.org/abs/2205.01068)

Fine-tune the OPT model on the dataset retrieved from [Goodreads Quotes](https://www.goodreads.com/quotes) suitable for multi-label text classification & text generation.

In [1]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-6.7b",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")
print(model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 4096, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
      (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
            (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
            (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
            (out_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear8bitLt(in_features=4096, out_features=16384, bias=True)
          (fc2): Linear8bitLt(in_features=16384, out_features=4096, bias=True

/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
import torch

# Postprocess the model
for param in model.parameters():
    param.requires_grad = False  # freeze the model - train adapters later
    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(torch.nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8388608 || all params: 6666862592 || trainable%: 0.12582542214183376


In [6]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='models/opt-6.7b-english-quotes-lora'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

2024-10-13 15:26:54.031661: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-13 15:26:54.038060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 15:26:54.046290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 15:26:54.048670: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-13 15:26:54.055993: I tensorflow/core/platform/cpu_feature_guar

Step,Training Loss
1,2.232600
2,2.406400
3,2.208900
4,2.227100
5,1.772600
6,2.360400
7,2.341000
8,2.201100
9,2.226000
10,2.384500


TrainOutput(global_step=200, training_loss=1.923757972717285, metrics={'train_runtime': 756.3886, 'train_samples_per_second': 4.231, 'train_steps_per_second': 0.264, 'total_flos': 1.164184860229632e+16, 'train_loss': 1.923757972717285, 'epoch': 1.2759170653907497})

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained('models/opt-6.7b-english-quotes-lora/checkpoint-200')
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, 'models/opt-6.7b-english-quotes-lora/checkpoint-200')
print(model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 4096, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
          (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (layers): ModuleList(
            (0-31): 32 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
                (v_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=16, bias=False)
                  )
                  (lora_B): Modul

In [7]:
batch = tokenizer("Two things are infinite: ", return_tensors='pt')

with torch.amp.autocast('cuda'):
    output_tokens = model.generate(**batch, max_new_tokens=50)

print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Two things are infinite:  the universe and human stupidity; and I'm not sure about the universe.  -Albert Einstein
I'm not sure about the universe either.


## 2-2. Prompt Tuning

# 3. Fine-Tune with Reinforcement Learning
**Alignment** instead of generating the most likely textual sequence, our language model learns to generate the textual sequence that is desired by a user.

Popular open source datasets for fine-tuning large language models (LLMs).
- The [Wikimedia Wikipedia](https://huggingface.co/datasets/wikimedia/wikipedia) dataset is built from the [Wikipedia Dumps](https://dumps.wikimedia.org/) with one subset per language, each containing a single train split.
- [The Pile: An 800GB Dataset of Diverse Text for Language Modeling](https://arxiv.org/abs/2101.00027)

## 3-1. Supervised Fine-Tuning (SFT)
1. `trl.SFTTrainer()`
2. `trl.SFTConfig`
    - By passing `packing=True` to `trl.SFTConfig`, multiple short examples are packed in the same input sequence to increase training efficiency. Set `eval_packing=False` to avoid packing the evaluation dataset. 

If you have a large dataset & need extensive customization for your training loop or complex training workflows, then go with `transformers.Trainer`; if you have a pre-trained model & a relatively smaller dataset and want a simpler & faster fine-tuning experience with efficient memory usage, then `trl.SFTTrainer` is a choice.

[Large Movie Review Dataset](https://huggingface.co/datasets/stanfordnlp/imdb) by Stanford.

In [4]:
from trl import SFTConfig, SFTTrainer

sft_config = SFTConfig(output_dir="models/gpt2-imdb")
print(sft_config)

SFTConfig(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
chars_per_token=<CHARS_PER_TOKEN>,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
dataset_batch_size=1000,
dataset_kwargs=None,
dataset_num_proc=None,
dataset_text_field=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=Tr

In [5]:
from transformers import AutoModelForCausalLM
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

dataset = load_dataset("stanfordnlp/imdb", split="train")

model = AutoModelForCausalLM.from_pretrained("gpt2")

sft_config = SFTConfig(output_dir="models/gpt2-imdb", dataset_text_field="text", max_seq_length=128)

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=sft_config,
)

trainer.train()

/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Step,Training Loss
500,3.779800
1000,3.706600
1500,3.686500
2000,3.657200
2500,3.638700
3000,3.619400
3500,3.509800
4000,3.479300
4500,3.477700
5000,3.482500


TrainOutput(global_step=9375, training_loss=3.512039765625, metrics={'train_runtime': 1038.4589, 'train_samples_per_second': 72.222, 'train_steps_per_second': 9.028, 'total_flos': 4899225600000000.0, 'train_loss': 3.512039765625, 'epoch': 3.0})

In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig

dataset = load_dataset("stanfordnlp/imdb", split="train")

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    "gpt2",
    train_dataset=dataset,
    args=SFTConfig(output_dir="models/gpt2-imdb-peft", dataset_text_field="text", max_seq_length=128),
    peft_config=peft_config
)

trainer.train()

/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:185: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/peft/tuners/lora/model.py:336: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step,Training Loss
500,3.896300


In [ ]:
# Adapters

### 3-1-1. Packing
**Example Packing** combines multiple sequences until a desired length and then feeds the model the packed batch without padding with special tokens. 
1. `ConstantLengthDataset`

In [1]:
# Pass `packing=True`
from transformers import AutoModelForCausalLM
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split="train")

model = AutoModelForCausalLM.from_pretrained("gpt2")

sft_config = SFTConfig(output_dir="models/gpt2-wikitext", packing=True, dataset_text_field="text", max_seq_length=128)

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=sft_config,
)

trainer.train()

2024-10-12 20:58:56.624705: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-12 20:58:56.632418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 20:58:56.641810: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 20:58:56.644702: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 20:58:56.652141: I tensorflow/core/platform/cpu_feature_guar

Generating train split: 0 examples [00:00, ? examples/s]

Step,Training Loss
500,3.712400
1000,3.572400
1500,3.511300
2000,3.470100
2500,3.410200
3000,3.266800
3500,3.280400
4000,3.271300
4500,3.249600
5000,3.194600


TrainOutput(global_step=7113, training_loss=3.3216946506808616, metrics={'train_runtime': 790.5149, 'train_samples_per_second': 71.957, 'train_steps_per_second': 8.998, 'total_flos': 3715768664064000.0, 'train_loss': 3.3216946506808616, 'epoch': 3.0})

### 3-1-2. Instruction Fine-Tuning

- [Finetuned Language Models Are Zero-Shot Learners](https://arxiv.org/abs/2109.01652)

The inputs of LLMs are called **Prompts** and outputs are referred to as **Completions**.

1. `trl.DataCollatorForCompletionOnlyLM(response_template, instruction_template, mlm=False, ignore_index=-100)`: Data collator used for **Instruction Masking** in completion tasks. Ensures that all the tokens of the template labels are set to  `ignore_index` so that the loss is only calculated on the completions only.
    - Pass an optional custom function, which groups & formats different role messages into a piece of text, to the `formatting_func` argument of `trl.SFTTrainer`, and then set `response_template`, like `### Assistant:\n`, & `instruction_template`, such as `### Human:\n`, in the data collator.
    - Can also use tokenized `input_ids` directly for the `response_template` & `instruction_template` arguments. This approach is convenient for LLMs with special tokenization rules like Llama.
    - https://github.com/huggingface/trl/blob/main/trl/trainer/utils.py

Train the model on the subset of the [OpenAssistant Conversations (OASST1)](https://huggingface.co/datasets/OpenAssistant/oasst1) dataset for example.
- [OpenAssistant Conversations - Democratizing Large Language Model Alignment](https://arxiv.org/abs/2304.07327)

In [1]:
# Completion only 
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
import warnings
warnings.filterwarnings("ignore")

dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

instruction_template = "### Human:"
response_template = "### Assistant:"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

trainer = SFTTrainer(
    model,
    args=SFTConfig(
        output_dir="models/opt-350m-oasst1",
        dataset_text_field = "text",
        max_seq_length=256
    ),
    train_dataset=dataset,
    data_collator=collator,
)

trainer.train()

2024-10-13 12:24:21.694691: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-13 12:24:21.808713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 12:24:21.855134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 12:24:21.869770: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-13 12:24:21.963873: I tensorflow/core/platform/cpu_feature_guar

Step,Training Loss
500,2.412800
1000,2.312900
1500,2.029300
2000,1.837200
2500,1.777600
3000,1.483900
3500,1.466100


/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/trl/trainer/utils.py:183: UserWarning: Could not find response key `### Assistant:` in the following instance: </s>### Human: Підсумуй цей текст (у результаті має вийти 5 інформативних речень):

Компанія SpaceX відзначилася черговою рекордною місією – ракета Falcon 9 вивела на орбіту найважчий для себе вантаж, доставивши в космос відразу 56 нових супутник This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(
/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/trl/trainer/utils.py:198: UserWarning: Could not find instruction key `### Human:` in the following instance: </s>### Human: Підсумуй цей текст (у результаті має вийти 5 інформативних речень):

Компанія SpaceX відзначилася черговою рекордною місією – ракета Falcon 9 вивела на орбіту найважчий для себе вантаж, доставивши в космос відразу 56 нових супутник This instance wi

TrainOutput(global_step=3693, training_loss=1.8801885254170687, metrics={'train_runtime': 1803.1942, 'train_samples_per_second': 16.381, 'train_steps_per_second': 2.048, 'total_flos': 1.3763345428512768e+16, 'train_loss': 1.8801885254170687, 'epoch': 3.0})

In [1]:
# Use the `formatting_func` argument
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
import warnings
warnings.filterwarnings("ignore")

dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=SFTConfig(output_dir="models/opt-350m-codealpaca"),
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    max_seq_length=256
)

trainer.train()

2024-10-13 18:28:02.162535: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-13 18:28:02.169879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 18:28:02.178544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 18:28:02.180936: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-13 18:28:02.187518: I tensorflow/core/platform/cpu_feature_guar

Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

Step,Training Loss
500,1.616400
1000,1.329300
1500,1.227500
2000,1.177400
2500,1.121100
3000,0.876900
3500,0.856300
4000,0.835000
4500,0.837800
5000,0.805100


TrainOutput(global_step=7509, training_loss=0.9071835766684471, metrics={'train_runtime': 3025.3616, 'train_samples_per_second': 19.854, 'train_steps_per_second': 2.482, 'total_flos': 2.2786043335213056e+16, 'train_loss': 0.9071835766684471, 'epoch': 3.0})

## 3-2. Reward Modeling (RM)

1. `trl.RewardTrainer`
2. `trl.RewardConfig`

`trl.RewardTrainer` expects a dataset on pairs of examples to predict which of the two is preferred, such as the [Anthropic HH-RLHF](https://huggingface.co/datasets/Anthropic/hh-rlhf) dataset.

3. `trl.RewardDataCollatorWithPadding`

ValueError: The features should include `input_ids_chosen`, `attention_mask_chosen`, `input_ids_rejected` and `attention_mask_rejected`


[Dataset Formats & Types](https://huggingface.co/docs/trl/main/en/dataset_formats#dataset-formats-and-types)

In [1]:
from datasets import load_dataset

dataset = load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model'],
        num_rows: 60917
    })
})


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import ModelConfig, RewardConfig, RewardTrainer, get_peft_config, get_quantization_config

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

2024-10-16 16:22:35.000330: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 16:22:35.007852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 16:22:35.015821: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 16:22:35.018200: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 16:22:35.025080: I tensorflow/core/platform/cpu_feature_guar

In [3]:
def preprocess_function(examples):
    new_examples = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": []
    }
    for chosen, rejected in zip(examples["chosen"], examples["rejected"]):
        tokenized_chosen = tokenizer(chosen[0]["content"], padding=True, truncation=True, max_length=256)
        tokenized_rejected = tokenizer(rejected[0]["content"], padding=True, truncation=True, max_length=256)

        new_examples["input_ids_chosen"].append(tokenized_chosen["input_ids"])
        new_examples["attention_mask_chosen"].append(tokenized_chosen["attention_mask"])
        new_examples["input_ids_rejected"].append(tokenized_rejected["input_ids"])
        new_examples["attention_mask_rejected"].append(tokenized_rejected["attention_mask"])

    return new_examples

dataset = dataset.map(preprocess_function, batched=True)
print(dataset)

Map:   0%|          | 0/60917 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 60917
    })
})


In [4]:
training_args = RewardConfig(
    output_dir="models/gpt2-ultrafeedback-binarized-preferences-cleaned"
)

trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"]
)

trainer.train()

/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/trl/trainer/reward_trainer.py:199: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
500,0.963000
1000,0.703800
1500,0.700500
2000,0.699200
2500,0.698500
3000,0.697000
3500,0.696800
4000,0.696600
4500,0.695700
5000,0.695900


/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and ther

TrainOutput(global_step=22845, training_loss=0.7009987590453327, metrics={'train_runtime': 7835.7557, 'train_samples_per_second': 23.323, 'train_steps_per_second': 2.915, 'total_flos': 0.0, 'train_loss': 0.7009987590453327, 'epoch': 3.0})

## 3-3. Proximal Policy Optimization (PPO)

[TRL's Sentiment and Descriptiveness Preference Dataset](https://huggingface.co/datasets/trl-internal-testing/descriptiveness-sentiment-trl-style)

In [ ]:
!python examples/scripts/ppo/ppo.py \
    --dataset_name trl-internal-testing/descriptiveness-sentiment-trl-style \
    --dataset_train_split descriptiveness \
    --learning_rate 3e-6 \
    --num_ppo_epochs 1 \
    --num_mini_batches 1 \
    --output_dir models/minimal/ppo \
    --per_device_train_batch_size 64 \
    --gradient_accumulation_steps 1 \
    --total_episodes 10000 \
    --model_name_or_path EleutherAI/pythia-1b-deduped \
    --missing_eos_penalty 1.0

# 4. Direct Preference Optimization (DPO)
- [Direct Preference Optimization: Your Language Model is Secretly a Reward Model](https://arxiv.org/abs/2305.18290)

# 5. Synthetic Data Generation
## 5-1. Knowledge Distillation

Knowledge Types:
- **Response-Based Knowledge:**
- **Feature-Based Knowledge:**
- **Relation-Based Knowledge:**

Distillation strategies:
- **Offline Distillation:**
- **Online Distillation:**
- **Self-Distillation:**

## 5-2. Self-Improvement

# 6. Model Pruning
Many parameters are redundant because they do not contribute much during training; therefore, such parameters can be removed from the network with little effect on accuracy after training. Pruned models sometimes outperform the original architecture but rarely outperform a better architecture.

- **Weight Pruning:** Weight connections that are below some predefined thresholds are pruned (zeroed out).
- **Neuron Pruning:** Prunes the neurons.
- **Filter Pruning:** Filters are ranked according to their importance, and the least important filters are removed from the network. The importance of the filters can be calculated by L1 or L2 norm.
- **Layer Pruning:** Layers can also be pruned.

# 7. Llama (Large Language Model Meta AI)

Access:
- [meta-llama/llama-models](https://github.com/meta-llama/llama-models) in GitHub
- [meta-llama/Meta-Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct) in Hugging Face

Information:
- [Comparison of Llama Models](https://en.wikipedia.org/wiki/Llama_(language_model)#Comparison_of_models)

Leaderboard:
- [Artificial Analysis](https://artificialanalysis.ai/)
- [LMSYS Chatbot Arena Leaderboard](https://chat.lmsys.org/?leaderboard)
- [Open LLM Leaderboard](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard)

## 7-1. Text Generation Web UI

Launch [Oobabooga Text Generation Web UI](https://github.com/oobabooga/text-generation-webui).

In [1]:
!huggingface-cli login --token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/yungshun317/.cache/huggingface/token
Login successful


In [ ]:
#@title 2. Launch the web UI

#@markdown If unsure about the branch, write "main" or leave it blank.

import os
from pathlib import Path

os.environ.pop('PYTHONPATH', None)

if Path.cwd().name != 'text-generation-webui':
  print("\033[1;32;1m\n --> Installing the web UI. This will take a while, but after the initial setup, you can download and test as many models as you like.\033[0;37;0m\n")

  !git clone https://github.com/oobabooga/text-generation-webui
  %cd text-generation-webui

  # Install the project in an isolated environment
  !GPU_CHOICE=A \
  USE_CUDA118=FALSE \
  LAUNCH_AFTER_INSTALL=FALSE \
  INSTALL_EXTENSIONS=FALSE \
  ./start_linux.sh

# Parameters
model_url = "https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct" #@param {type:"string"}
branch = "main" #@param {type:"string"}
command_line_flags = "--n-gpu-layers 128 --load-in-4bit --use_double_quant --no_flash_attn" #@param {type:"string"}
api = False #@param {type:"boolean"}

if api:
  for param in ['--api', '--public-api']:
    if param not in command_line_flags:
      command_line_flags += f" {param}"

model_url = model_url.strip()
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

    # Download the model
    url_parts = model_url.strip('/').strip().split('/')
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    branch = branch.strip('"\' ')
    if branch.strip() not in ['', 'main']:
        output_folder += f"_{branch}"
        !python download-model.py {model_url} --branch {branch}
    else:
        !python download-model.py {model_url}
else:
    output_folder = ""

# Start the web UI
cmd = f"./start_linux.sh {command_line_flags} --share"
if output_folder != "":
    cmd += f" --model {output_folder}"

!$cmd


 --> Installing the web UI. This will take a while, but after the initial setup, you can download and test as many models as you like.

Cloning into 'text-generation-webui'...
remote: Enumerating objects: 18858, done.
remote: Counting objects: 100% (492/492), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 18858 (delta 349), reused 384 (delta 275), pack-reused 18366
Receiving objects: 100% (18858/18858), 27.83 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (13355/13355), done.
/content/text-generation-webui
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 69.7M  100 69.7M    0     0   286M      0 --:--:-- --:--:-- --:--:--  288M
PREFIX=/content/text-generation-webui/installer_files/conda
Unpacking payload ...
                                                                                             
Installing base environment...





Preparing tra

## 7-2. LangChain Hugging Face

Llama 3 8B requires around 16GB of disk space and 20GB of VRAM (GPU memory) in FP16. As for Llama 3 70B, it requires around 140GB of disk space and 160GB of VRAM in FP16. Take AWS for example, you must have at least `g5.2xlarge` for Llama 3 8b and `g5.12xlarge` for Llama 3 70b. 

- [HuggingChat](https://huggingface.co/chat/) for using community's best chat models.

In [ ]:
# Get models from Meta
# !huggingface-cli download meta-llama/Meta-Llama-3.1-8B-Instruct --include "original/*" --local-dir models/meta-llama/Meta-Llama-3.1-8B-Instruct

In [2]:
# !pip install langchain langchain_community

# `rope_scaling` must be a dictionary with two fields, `type` and `factor`, got {'factor': 8.0, 'low_freq_factor': 1.0, 'high_freq_factor': 4.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'}
# !pip install --upgrade transformers
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')

In [10]:
# 4-bit quantization
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
nf4_model = transformers.AutoModelForCausalLM.from_pretrained(model, quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=nf4_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    repetition_penalty=1.1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={ "temperature": 0.7, "stop": ["<|eom_id|>", "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>", "<|reserved_special_token"]})

In [12]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x78b16b3320f0>, model_kwargs={'temperature': 0.7, 'stop': ['<|eom_id|>', '<|start_header_id|>', '<|end_header_id|>', '<|eot_id|>', '<|reserved_special_token']})

In [16]:
# Imbued prompt template
response = llm.invoke("""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>


Environment: ipython
Tools: brave_search, wolfram_alpha

Cutting Knowledge Date: 01 March 2023
Today's Date: 13 July 2024


You are a helpful Assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Weather in Menlo Park, California<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")
response

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'\n<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n\nEnvironment: ipython\nTools: brave_search, wolfram_alpha\n\nCutting Knowledge Date: 01 March 2023\nToday\'s Date: 13 July 2024\n\n\nYou are a helpful Assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWeather in Menlo Park, California<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nbrave_search.call(query="Menlo Park, California weather")assistant\n\nThe current weather in Menlo Park, California is not available to me.  I can suggest some ways for you to find the current weather in Menlo Park, California.\n\nYou can check online weather websites such as AccuWeather or Weather.com for the current weather conditions and forecast in Menlo Park, California.\n\nAlternatively, you can also check the National Weather Service (NWS) website for the current weather conditions and forecast in Menlo Park, California.\n\nIf you want to get the current weather conditions and forecast in Menlo Park, California, I

In [17]:
print(response)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>


Environment: ipython
Tools: brave_search, wolfram_alpha

Cutting Knowledge Date: 01 March 2023
Today's Date: 13 July 2024


You are a helpful Assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Weather in Menlo Park, California<|eot_id|><|start_header_id|>assistant<|end_header_id|>
brave_search.call(query="Menlo Park, California weather")assistant

The current weather in Menlo Park, California is not available to me.  I can suggest some ways for you to find the current weather in Menlo Park, California.

You can check online weather websites such as AccuWeather or Weather.com for the current weather conditions and forecast in Menlo Park, California.

Alternatively, you can also check the National Weather Service (NWS) website for the current weather conditions and forecast in Menlo Park, California.

If you want to get the current weather conditions and forecast in Menlo Park, California, I recommend checking thes

## 7-3. Fine-Tune Llama 2 with PyTorch
## 7-4. Fine-Tune Llama 2 with TRL


# 8. Multimodal Learning
## 8-1. Fine-Tune LLaVA with TRL

- [LLaVA: Large Language and Vision Assistant](https://llava-vl.github.io/)
- [Visual Instruction Tuning](https://arxiv.org/abs/2304.08485)